<a href="https://colab.research.google.com/github/LuanHuamani84/practica1/blob/main/Latitud_longitud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Importar librerias
import pandas as pd
import numpy as np
from unicodedata import normalize

In [2]:
## Parte 1. Extracción de datos de la página de wikipedia
tablas=pd.read_html('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M')
postales = tablas[0]
post_df = pd.DataFrame({'datos':[]})
k = 0
for x in range(9):
    for j in range(20):
        post_df.loc[k] = postales[x][j]
        k = k+1
post_df['datos'] = post_df['datos'].str.replace('/',',')
post_df['datos'] = post_df['datos'].str.replace(')','')
post_df = post_df.assign(PostalCode='6')
post_df = post_df.assign(Borough_='6')

for x in range(180):
    post_df['PostalCode'][x], post_df['Borough_'][x] = post_df['datos'][x][:3], post_df['datos'][x][3:]

separado = post_df['Borough_'].str.split('(',expand=True)
separado.columns = ['Borough','Neighborhood','tres']
post_df = pd.concat([post_df, separado], axis=1)
post_df = post_df.drop(['datos','Borough_','tres'], axis = 1)
post_df['Neighborhood'].replace([None], np.nan, inplace=True)
post_df = post_df[post_df.Borough != 'Not assigned']
post_df.reset_index(drop=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  # This is added back by InteractiveShellApp.init_path()


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village , St. Phillips , Martin Grov..."
101,M9V,Etobicoke,"South Steeles , Silverstone , Humbergate , Jam..."


In [3]:
# Se desacarga el archivo con las coordenadas
!wget -q -O 'Geospatial_Coordinates.csv'  http://cocl.us/Geospatial_data

In [4]:
# Se lee el archivo csv
coord_df = pd.read_csv('Geospatial_Coordinates.csv')
coord_df

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [5]:
# Concatenar los dos frames
post_coor = pd.merge(post_df, coord_df, left_on='PostalCode', right_on='Postal Code')
post_coor = post_coor.drop(['Postal Code'], axis=1)
post_coor

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village , St. Phillips , Martin Grov...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles , Silverstone , Humbergate , Jam...",43.739416,-79.588437
